In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import math
import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import tensorflow_hub as hub
import h5py
from sklearn import metrics
from tensorflow.keras import regularizers

# !wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
import tokenization

import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd

2021-12-14 12:24:33.356895: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2
/home/rajic/miniconda3/envs/ada/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: mpi4py.MPI.Status size changed, may indicate binary incompatibility. Expected 40 from C header, got 48 from PyObject
  return f(*args, **kwds)
/home/rajic/miniconda3/envs/ada/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: mpi4py.MPI.Datatype size changed, may indicate binary incompatibility. Expected 24 from C header, got 32 from PyObject
  return f(*args, **kwds)
/home/rajic/miniconda3/envs/ada/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: mpi4py.MPI.Request size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  return f(*args, **kwds)
/home/rajic/miniconda3/envs/ada/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: mpi4py.MPI.Prequest size changed, may indicate binary inco

In [22]:
MODEL_CHECKPOINT_NAME = "model_lr=5e-06_hid=1024,512,256_maxlen=300_batch=8_epochs=100_seed=72_l2=0.003.h5"
MAX_LEN = 300  # Needs to match the checkpoint above!

n = 234994989
BATCH_SIZE = 8
SAMPLE_PICKLE = "quotebank-sample.pickle"
SAMPLE_BERT_RESULTS_PICKLE = "quotebank-sample-bert-evaluation.pickle"
SAMPLE_BERT_RESULTS_DF = "quotebank-sample-bert-evaluation.csv"
SAMPLE_BERT_RESULTS_DF_DISASTERS_ONLY = "quotebank-sample-bert-evaluation-disasters-only.csv"
DATASET_PATH = "../quotebank"

In [3]:
# fetch bert layer
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

# initialize tokenizer
tokenizer = tokenization.FullTokenizer(
    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy(),
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
)

print(f"BERT layer to be used loaded")

2021-12-14 12:24:50.090155: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-12-14 12:24:50.117478: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:86:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-12-14 12:24:50.117517: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2
2021-12-14 12:24:50.123950: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-12-14 12:24:50.129758: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2021-12-14 12:24:50.130339: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcurand.so

BERT layer to be used loaded


In [4]:
model = load_model(MODEL_CHECKPOINT_NAME, custom_objects={'KerasLayer': hub.KerasLayer})

print(f"Loaded checkpoint: {MODEL_CHECKPOINT_NAME}")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Loaded checkpoint: model_lr=5e-06_hid=1024,512,256_maxlen=300_batch=8_epochs=100_seed=72_l2=0.003.h5


In [5]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

texts_sample = [
    "This is a disaster!",
    "But this is not!",
    "Haiti earthquake 2010!",
    "lava ground!"]
x = bert_encode(texts_sample, tokenizer, max_len=MAX_LEN)
model.predict(x).round().astype(np.uint8)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-12-14 12:25:34.446566: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


array([[0],
       [0],
       [1],
       [0]], dtype=uint8)

In [6]:
from dask.diagnostics import ProgressBar
pbar = ProgressBar()                
pbar.register()

In [7]:
def predict(model, texts_batch):
    batch_input = bert_encode(texts_batch, tokenizer, max_len=MAX_LEN)
    predictions = model.predict(batch_input).round().astype(np.uint8)
    return predictions.squeeze() # np.array([0,1,1,0,0,0,...])

@dask.delayed
def predict_delayed(model, texts_batch):
    batch_input = bert_encode(texts_batch, tokenizer, max_len=MAX_LEN)
    predictions = model.predict(batch_input).round().astype(np.uint8)
    return predictions.squeeze() # np.array([0,1,1,0,0,0,...])

z = predict_delayed(model, texts_sample)

with pbar:
    print(z)
    print(z.compute())

Delayed('predict_delayed-740617ef-ed8b-46f9-ab87-ff1d146e7f5b')
[########################################] | 100% Completed |  0.2s
[0 0 1 0]


In [8]:
with pbar:
    array = da.from_delayed(z, (4,), dtype=np.uint8)
    print(array)
    print(array.compute())

dask.array<from-value, shape=(4,), dtype=uint8, chunksize=(4,), chunktype=numpy.ndarray>
[########################################] | 100% Completed |  0.2s
[0 0 1 0]


In [9]:
# Feels too slow
#
# with pbar:
#     array = da.from_delayed([predict_delayed(model, [q]) for q in dataset.quotation.str], (n,), dtype=np.uint8)
#     print(array)
#     array.compute()
# print(array[:10])

In [10]:
# Also too slow, stayed at 0% after ?? minutes
# The limit I have for the machine is 24h, so one
# percent could take at most about 14 minutes
# 
# with pbar:
#     dataset = dataset.repartition(npartitions=n//8+1)
#     dataset["disaster"] = dataset.map_partitions(lambda x: predict(model, x["quotation"]), meta="np.uint8")
#     disasters = dataset["disaster"].compute()

In [11]:
sample = pd.read_pickle(SAMPLE_PICKLE)
# sample = sample.head(32)
sample_dask = dd.from_pandas(sample, npartitions=len(sample)//8 + 1)
sample_dask.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,urls,phase,speaker_prob,url_countries
0,2011-05-14-019322,"in my opinion, we're just like a lot of compan...",david wilson,Multiple,2011-05-14 21:54:59,1,[http://rockymounttelegram.com/business/five-b...,C,0.5312,[US]
0,2020-03-30-014125,Has some some form of recognizable dementia.,Tucker Carlson,Q743162,2020-03-30 22:17:44,1,[http://breitbart.com/politics/2020/03/30/pink...,E,0.6052,[US]
1,2009-05-04-003681,and thus your punishment is cleverly and autom...,david eagleman,Q999577,2009-05-04 01:09:42,1,[http://ledger-enquirer.com/392/story/705488.h...,A,0.6456,[US]
2,2018-03-06-054089,"I'd normally hate these sort of shots, but I l...",None,None,2018-03-06 17:58:24,2,[http://www.thefashionspot.com/buzz-news/forum...,E,0.8656,[US]
4,2014-12-22-066977,we are so caught up in our thoughts and concer...,Fouad Twal,Q124679,2014-12-22 05:00:12,1,[http://www.catholicherald.co.uk/news/2014/12/...,E,0.9351,[UK]


In [12]:
with pbar:
    sample["disaster"] = sample_dask.map_partitions(lambda x: predict(model, x["quotation"]), meta="np.uint8").compute()
    print(f"Sample: {sum(sample.disaster)} disasters out of {len(sample)}")

sample_disasters = sample[sample.disaster == 1]
sample_disasters.head()

[##################                      ] | 47% Completed | 44min  0.9s

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[##########################              ] | 67% Completed |  1hr  2min 47.6s

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[########################################] | 100% Completed |  1hr 33min 25.6s
Sample: 22893 disasters out of 234991


,quoteID,quotation,speaker,qids,date,numOccurrences,urls,phase,speaker_prob,url_countries,disaster
51940,2008-10-23-057784,"they're running up to play run defense, and we...",troy calhoun,Q7846785,2008-10-23 03:31:14,1,[http://gazette.com/sports/option_42273___arti...,A,0.5571,[US],1
34659,2008-10-29-014667,"however, i knew athol from our shared passion ...",colin mcmillan,Multiple,2008-10-29 04:06:30,1,[http://theguardian.pe.ca/index.cfm?sid=184597...,A,0.7189,[CA],1
36313,2008-11-24-061382,why did kelly and jessica have to die? it's sa...,matt mitchell,Multiple,2008-11-24 10:41:27,1,[http://bnd.com/homepage/story/552938.html],A,0.5843,[US],1
70031,2008-09-27-037709,to think someone was taking money at a time wh...,linda berglin,Q13562381,2008-09-27 04:55:22,2,[http://startribune.com/politics/state/2980187...,A,0.7615,[US],1
77381,2008-12-12-001887,all parts of the turner entertainment networks...,michael wright,Multiple,2008-12-12 05:14:36,2,[http://hollywoodreporter.com/hr/content_displ...,A,0.7261,[US],1


In [23]:
import pickle as pickle
with open(SAMPLE_BERT_RESULTS_PICKLE, 'wb') as f:
    pickle.dump(sample, f, protocol=pickle.HIGHEST_PROTOCOL)
    
sample.to_csv(SAMPLE_BERT_RESULTS_DF, encoding='utf-8')
sample_disasters.to_csv(SAMPLE_BERT_RESULTS_DF_DISASTERS_ONLY, encoding='utf-8')

In [17]:
sample = pd.read_pickle(SAMPLE_BERT_RESULTS_PICKLE)
sample_disasters = sample[sample.disaster == 1]
sample_disasters.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,urls,phase,speaker_prob,url_countries,disaster
51940,2008-10-23-057784,"they're running up to play run defense, and we...",troy calhoun,Q7846785,2008-10-23 03:31:14,1,[http://gazette.com/sports/option_42273___arti...,A,0.5571,[US],1
34659,2008-10-29-014667,"however, i knew athol from our shared passion ...",colin mcmillan,Multiple,2008-10-29 04:06:30,1,[http://theguardian.pe.ca/index.cfm?sid=184597...,A,0.7189,[CA],1
36313,2008-11-24-061382,why did kelly and jessica have to die? it's sa...,matt mitchell,Multiple,2008-11-24 10:41:27,1,[http://bnd.com/homepage/story/552938.html],A,0.5843,[US],1
70031,2008-09-27-037709,to think someone was taking money at a time wh...,linda berglin,Q13562381,2008-09-27 04:55:22,2,[http://startribune.com/politics/state/2980187...,A,0.7615,[US],1
77381,2008-12-12-001887,all parts of the turner entertainment networks...,michael wright,Multiple,2008-12-12 05:14:36,2,[http://hollywoodreporter.com/hr/content_displ...,A,0.7261,[US],1
